In [1]:
# set module auto-reloaded 
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman
import pickle
from Compressible_Huffman import CompressibleNN



In [13]:
# Load the CompressibleNN instance
with open('compressed_nn.pkl', 'rb') as inp:
    net_model = pickle.load(inp)
    codec = pickle.load(inp)
    compressed_model_weights = pickle.load(inp)

# Create a new instance of CompressibleNN
compNN = CompressibleNN(net_model)
compNN.codec = codec

In [14]:
decompressed_weights = compNN.decompressNN(compressed_model_weights)

In [15]:
# Compare the original weights with the decompressed weights
with open('original_model_weights.pkl', 'rb') as file:
    original_model_weights = pickle.load(file)

differences = compNN.compare_weights(original_model_weights, decompressed_weights)

print("Differences between original and decompressed weights:")
for i, diff in enumerate(differences):
    print(f"Layer {i+1}: Max Difference = {np.max(diff)}, Mean Difference = {np.mean(diff)}")


Differences between original and decompressed weights:
Layer 1: Max Difference = 0.0, Mean Difference = 0.0
Layer 2: Max Difference = 0.0, Mean Difference = 0.0
Layer 3: Max Difference = 0.0, Mean Difference = 0.0
Layer 4: Max Difference = 0.0, Mean Difference = 0.0
Layer 5: Max Difference = 0.0, Mean Difference = 0.0
Layer 6: Max Difference = 0.0, Mean Difference = 0.0


In [17]:
print(net_model.get_weights())

[array([[-0.01356051, -0.02920462, -0.08238873, ...,  0.08018555,
         0.06390834, -0.06385274],
       [ 0.05425084,  0.01795489, -0.0678107 , ...,  0.01726173,
        -0.02113655, -0.02320373],
       [-0.01574795, -0.0846318 , -0.10405112, ...,  0.08676577,
         0.06584985,  0.05149375],
       ...,
       [-0.00997006, -0.02281972, -0.06760016, ..., -0.05238574,
         0.02308347, -0.00087993],
       [ 0.03146753,  0.08941627, -0.05239892, ..., -0.03657151,
         0.01679081,  0.03568301],
       [-0.01346833,  0.05070372,  0.04047205, ...,  0.05235921,
        -0.05342254,  0.08456092]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [18]:
print(decompressed_weights)

[array([[-0.01356051, -0.02920462, -0.08238873, ...,  0.08018555,
         0.06390834, -0.06385274],
       [ 0.05425084,  0.01795489, -0.0678107 , ...,  0.01726173,
        -0.02113655, -0.02320373],
       [-0.01574795, -0.0846318 , -0.10405112, ...,  0.08676577,
         0.06584985,  0.05149375],
       ...,
       [-0.00997006, -0.02281972, -0.06760016, ..., -0.05238574,
         0.02308347, -0.00087993],
       [ 0.03146753,  0.08941627, -0.05239892, ..., -0.03657151,
         0.01679081,  0.03568301],
       [-0.01346833,  0.05070372,  0.04047205, ...,  0.05235921,
        -0.05342254,  0.08456092]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,